# End-to-End Analysis of Kaggle Airbnb Data
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/carriegardner428/ML-with-Python-Tepper-CY21-AW4/blob/main/notebooks/Introduction.ipynb)

## Set Up
First, let's import our dependencies into the notebook.  We are using numpy and pandas to represent data in a tabular format, we are using sklearn and statsmodels for modeling, and matplotlib and plotly for visiualizations.

In [ ]:
# Data Representation
import numpy as np
import pandas as pd

# Modeling
import sklearn
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version
from sklearn.tree import export_graphviz
import statsmodels

# Visualization
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Image
import plotly
import plotly.express as px

# Magic Commands
%matplotlib inline

random_state = 42

## Collect the Data

In [ ]:
link = 'https://drive.google.com/file/d/1tT0lNiDHwGQPLa3N0zSdhZcJZaa5aqF3/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+link.split('/')[-2]

In [ ]:
listings_df = pd.read_csv(path)
listings_df.head(1)

## Get to Know the Data with Exploratory Data Analysis (EDA)

In [ ]:
listings_df.info()

In [ ]:
listings_df["property_type"].value_counts()

In [ ]:
listings_df.select_dtypes(exclude='object')

In [ ]:
listings_df.select_dtypes(include='object').columns

In [ ]:
listings_df['room_type'].value_counts()

In [ ]:
listings_df.select_dtypes(exclude='object').columns

In [ ]:
non_object_columns = listings_df.select_dtypes(exclude='object').columns
non_object_columns

In [ ]:
listings_df['bathrooms'].astype('object')

In [ ]:
non_object_columns = [column for column in non_object_columns if not 'id' in column or not 'availability' in column]
non_object_columns

In [ ]:
listings_df[non_object_columns].describe()

In [ ]:
listings_df.describe()

In [ ]:
px.histogram(listings_df, x="bedrooms", title="Histogram of Bedrooms", labels={'bedrooms':"Number of Bedrooms"})

In [ ]:
beds_baths = listings_df[['bedrooms', 'bathrooms']]
beds_baths['bathrooms'] = beds_baths['bathrooms'].astype('object')

In [ ]:
px.histogram(beds_baths.dropna(), x="bedrooms", color="bathrooms", nbins=5)

In [ ]:
px.histogram(beds_baths.dropna(), x="bathrooms", color="bedrooms", nbins=5, title="Histogram of Bedrooms with Bathrooms",)

In [ ]:
px.histogram(listings_df, x="price", title="Histogram of Listing Prices", labels={'price':"Price Per Night in $"}, nbins=20)

In [ ]:
### Split 

In [ ]:
listings_df.shape

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(listings_df, test_size=0.2, random_state=random_state)

In [ ]:
train_set.shape

test_set.shape

## Discover and Visualize the Data

In [ ]:
listings = train_set.copy()

In [ ]:
px.scatter(listings, x="longitude", y="latitude")

In [ ]:
px.scatter_geo(listings, 
                lat="latitude", 
                lon="longitude", 
                #scope='usa', 
                projection="natural earth",
                center={'lat':listings['latitude'].mean(), 'lon':listings['longitude'].mean()}
                )

In [ ]:
listings['latitude'].mean()

In [ ]:
px.scatter_mapbox(listings, 
                lat="latitude", 
                lon="longitude", 
                #scope='usa', 
                #projection="natural earth",
                mapbox_style='open-street-map',
                zoom=9,
                center={'lat':listings['latitude'].mean(), 'lon':listings['longitude'].mean()}
                )

In [ ]:
listings['price'].dtype

In [ ]:
listings['price'].iloc[0] # get first item in the price column

In [ ]:
listings['price'].sort_values().iloc[0:10] # get first ten items in the price column sorted from greatest to least

> We see that the prices are strings and contain the characters: "$", ",", and "."

> If we would like to treat this feature as a continuous variable, we will need to clean it

In [ ]:
listings['price_float'] = listings['price'].replace('[\$\,]',"",regex=True).astype(float)
listings['price_float'].dtype

In [ ]:
px.scatter_mapbox(listings.sort_values(by=['price_float'],ascending=False).head(500), 
                lat="latitude", 
                lon="longitude", 
                mapbox_style='carto-positron',
                center={'lat':listings['latitude'].mean(), 'lon':listings['longitude'].mean()},
                color="price_float",
                size="accommodates",
                zoom=11,
                size_max=10,
                color_continuous_scale=px.colors.cyclical.IceFire,
                hover_data=['bedrooms', 'bathrooms', 'price'],
                title="Listing Locations by Price"
                )

In [ ]:
corr_matrix = listings.corr()

In [ ]:
corr_matrix["price_float"].sort_values(ascending=False)

In [ ]:
listings['accommodates']

In [ ]:
listings['square_feet']

In [ ]:
px.scatter(listings, x="price", y="bedrooms", title="Price vs. Bedrooms")

In [ ]:
listings['price_float'].head()

In [ ]:
px.scatter_matrix(listings, 
    dimensions=['accommodates', 'bathrooms','bedrooms', 'beds','price'], 
    )

In [ ]:
help(px.parallel_categories)

In [ ]:
px.parallel_categories(listings.sort_values(by=['price_float'], ascending=False).head(25), 
                        dimensions=['accommodates', 'bathrooms','bedrooms', 'beds', 'zipcode'],
                        color='price_float', 
                        color_continuous_scale=px.colors.sequential.Inferno
                        )

## Prepare the Data for Machine Learning

In [ ]:
train_set.columns

In [ ]:
listings = train_set.drop(["price"], axis=1) # drop target labels for training set (price and price float)
listings_labels = train_set[["price"]].copy()
listings_labels['price_float'] = train_set['price'].replace('[\$\,]',"",regex=True).astype(float)

In [ ]:
sample_incomplete_rows = listings[listings.isnull().any(axis=1)]
sample_incomplete_rows.shape

### Impute Missing Data

## Select & Fit a Model

### Linear Regression w/`statsmodels`

In [ ]:
import statsmodels.api as sm

In [ ]:
olsmod = sm.OLS(y2,x2)
olsres = olsmod.fit()

In [ ]:
print(olsres.summary())

In [ ]:
px.scatter_mapbox(listings, 
                lat="latitude", 
                lon="longitude", 
                #scope='usa', 
                #projection="natural earth",
                mapbox_style='basic',
                center={'lat':listings['latitude'].mean(), 'lon':listings['longitude'].mean()}
                )

In [ ]:
px.scatter_mapbox(listings, 
                lat="latitude", 
                lon="longitude", 
                #scope='usa', 
                #projection="natural earth",
                center={'lat':listings['latitude'].mean(), 'lon':listings['longitude'].mean()}
                )